<a href="https://colab.research.google.com/github/bhadreshpsavani/NLP-Notes/blob/master/NaturalQAT5BaselineModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
!git clone https://github.com/efficientqa/nq-open.git

Cloning into 'nq-open'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 0), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [2]:
!pip install transformers -q
!pip install wandb -q

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 890kB 10.9MB/s 
     |████████████████████████████████| 1.1MB 23.7MB/s 
     |████████████████████████████████| 3.0MB 32.5MB/s 
     |████████████████████████████████| 1.4MB 3.5MB/s 
     |████████████████████████████████| 163kB 20.9MB/s 
     |████████████████████████████████| 102kB 7.5MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 112kB 18.3MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

Thu Aug  6 06:05:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

print(device)

cuda


## Data Exploration

In [6]:
train_df = pd.read_json("nq-open/NQ-open.train.jsonl", orient='columns', lines=True)
train_df.head()

,answer,question
0,[Fernie Alpine Resort],where did they film hot tub time machine
1,[Neither vessel],who has the right of way in international waters
2,[Marley],who does annie work for attack on titan
3,"[November 6, 1986]",when was the immigration reform and control ac...
4,[1950],when was puerto rico added to the usa


In [7]:
dev_df = pd.read_json("nq-open/NQ-open.dev.jsonl", orient='columns', lines=True)
dev_df.head()

,question,answer
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]"
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]"
2,how many seasons of the bastard executioner ar...,"[one, one season]"
3,when did the eagles win last super bowl,[2017]
4,who won last year's ncaa women's basketball,[South Carolina]


In [8]:
number_of_answers = pd.Series([len(train_df['answer'][i]) for i in range(len(train_df['answer']))])
number_of_answers.value_counts()

1     79300
2      4488
3      1710
4       895
5       512
6       310
7       245
8       175
10      137
9       132
12       12
11        3
21        2
13        1
15        1
18        1
25        1
dtype: int64

In [9]:
train_df['number_of_answers']=number_of_answers
train_df.head()

,answer,question,number_of_answers
0,[Fernie Alpine Resort],where did they film hot tub time machine,1
1,[Neither vessel],who has the right of way in international waters,1
2,[Marley],who does annie work for attack on titan,1
3,"[November 6, 1986]",when was the immigration reform and control ac...,1
4,[1950],when was puerto rico added to the usa,1


## Data Preprocessing

In [10]:
sample_questions = train_df['question'].head()
sample_answers = train_df['answer'].head()
sample_answers

0    [Fernie Alpine Resort]
1          [Neither vessel]
2                  [Marley]
3        [November 6, 1986]
4                    [1950]
Name: answer, dtype: object

In [11]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [12]:
print(sample_questions[0])
tokenizer.encode(sample_questions[0])

where did they film hot tub time machine


[213, 410, 79, 814, 1312, 6244, 97, 1437]

In [13]:
tokenizer.sep_token="<sep>"
tokenizer.sep_token

'<sep>'

In [14]:
sample_answers[4]

['1950']

In [15]:
multi_answers = train_df.query('number_of_answers>1')['answer']
multi_answers.head()

7     [Mangaung Metropolitan Municipality, Nelson Ma...
13                           ["Buzz" Brainard, Cameron]
16                        [Sadie Sandler, Selena Gomez]
17                                  [768 (HD), 68 (SD)]
20            [Martin Peters, Geoff Hurst, Bobby Moore]
Name: answer, dtype: object

## DataProcessing

In [16]:
tokenizer.sep_token="<sep>"
tokenizer.sep_token

'<sep>'

In [107]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.question = self.data.question
        self.answer = self.data.answer

    def __len__(self):
        return len(self.question)

    def __getitem__(self, index):
        question = str(self.question[index])
        question = 'trivia question: '+' '.join(question.split())+'? </s>'
        answer = ' '.join(self.answer[index]) + " </s>"
        answer = ' '.join(answer.split())
        
        print(question, " : ", answer)
        # print(question,":",answer)
        
        source = self.tokenizer.batch_encode_plus(
            [question], 
            max_length= self.source_len,
            add_special_tokens=True,
            return_special_tokens_mask=True,
            truncation=True, 
            pad_to_max_length=True,
            return_tensors='pt')
        
        target = self.tokenizer.batch_encode_plus(
            [answer], 
            max_length= self.source_len, 
            add_special_tokens=True,
            return_special_tokens_mask=True,
            truncation=True,
            pad_to_max_length=True,
            return_tensors='pt')
        
        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

## Training and Validation

In [108]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)
        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [109]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)
                
            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                repetition_penalty=5.0, 
                length_penalty=2.0, 
                early_stopping=True
                )
            
            preds = [tokenizer.decode(g) for g in generated_ids]
            target = [tokenizer.decode(t) for t in y]
            
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [110]:
TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 2
# number of epochs to train (default: 10)
VAL_EPOCHS = 1 
LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
SEED = 42               # random seed (default: 42)
MAX_LEN = 512

# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(SEED) # pytorch random seed
np.random.seed(SEED) # numpy random seed
torch.backends.cudnn.deterministic = True

# tokenzier for encoding the text
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [111]:
# Creation of Dataset and Dataloader
# Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
train_size = 0.95
train_df = pd.read_json("nq-open/NQ-open.train.jsonl", orient='columns', lines=True)
train_df = train_df[:5000]
train_dataset=train_df.sample(frac=train_size, random_state = SEED).reset_index(drop=True)
val_dataset=train_df.drop(train_dataset.index).reset_index(drop=True)
val_dataset = val_dataset
print("FULL Dataset: {}".format(train_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

FULL Dataset: (5000, 2)
TRAIN Dataset: (4750, 2)
TEST Dataset: (250, 2)


In [112]:
# Creating the Training and Validation dataset for further creation of Dataloader
training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN)

# Defining the parameters for creation of dataloaders
train_params = {
    'batch_size': TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

# Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

In [113]:
# for itm in training_loader:
#     print()

In [116]:
for itm in val_loader:
    print(itm)
    break

trivia question: where is this year's world cup being held? </s>  :  Russia </s>
trivia question: where was the first motte and bailey castle built? </s>  :  northern Europe </s>
{'source_ids': tensor([[22377,   822,    10,  ...,     0,     0,     0],
        [22377,   822,    10,  ...,     0,     0,     0]]), 'source_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'target_ids': tensor([[4623,    1,    0,  ...,    0,    0,    0],
        [8390, 1740,    1,  ...,    0,    0,    0]]), 'target_ids_y': tensor([[4623,    1,    0,  ...,    0,    0,    0],
        [8390, 1740,    1,  ...,    0,    0,    0]])}


In [105]:
# Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = T5ForConditionalGeneration.from_pretrained("t5-small")
model = model.to(device)

# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [106]:
#Training loop
print('Initiating Fine-Tuning for the model on our dataset')

for epoch in range(TRAIN_EPOCHS):
    train(epoch, tokenizer, model, device, training_loader, optimizer)

Initiating Fine-Tuning for the model on our dataset
Epoch: 0, Loss:  7.596241474151611
Epoch: 0, Loss:  2.5164263248443604
Epoch: 0, Loss:  3.5921876430511475
Epoch: 0, Loss:  1.9035779237747192
Epoch: 0, Loss:  3.5881741046905518
Epoch: 1, Loss:  2.7576749324798584
Epoch: 1, Loss:  3.7929913997650146
Epoch: 1, Loss:  3.9594757556915283
Epoch: 1, Loss:  4.681192398071289
Epoch: 1, Loss:  3.077524423599243


In [117]:
# Validation loop and saving the resulting file with predictions and acutals in a dataframe.
# Saving the dataframe as predictions.csv
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})

Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe
trivia question: where is this year's world cup being held? </s>  :  Russia </s>
trivia question: where was the first motte and bailey castle built? </s>  :  northern Europe </s>
Completed 0
trivia question: who won the intercontinental championship on monday night raw? </s>  :  The Miz </s>
trivia question: the great and small cardiac veins deliver their blood to what vessel? </s>  :  coronary sinus </s>
trivia question: who are the architects the athenian acropolis and who was in charge of the sculptural program? </s>  :  Phidias Pericles </s>
trivia question: what was julie's job on the love boat? </s>  :  Cruise Director </s>
trivia question: who won the womens world cup in 2015? </s>  :  the United States </s>
trivia question: what language do they speak in the country of georgia? </s>  :  Georgian </s>
trivia question: which place in india receives highest rainfall in the worl

In [118]:
 final_df.to_csv('predictions.csv')

In [119]:
for i in range(10):
    print("Actual Answer: ", final_df['Actual Text'][i],"\nPredicted Answer: ", final_df['Generated Text'][i])
    print()

Actual Answer:  Russia 
Predicted Answer:  New York

Actual Answer:  northern Europe 
Predicted Answer:  the castle

Actual Answer:  The Miz 
Predicted Answer:  John Williams

Actual Answer:  coronary sinus 
Predicted Answer:  the heart

Actual Answer:  Phidias Pericles 
Predicted Answer:  the architect

Actual Answer:  Cruise Director 
Predicted Answer:  julie

Actual Answer:  the United States 
Predicted Answer:  the womens

Actual Answer:  Georgian 
Predicted Answer:  georgia

Actual Answer:  Cherrapunji 
Predicted Answer:  India

Actual Answer:  TLC 
Predicted Answer:  John

